# Principal Component Analysis for Linear Dimensionality Reduction & Multi-class classification Using PathMNIST dataset

The number of features or variables used in a dataset is also known as the dimensionality. Often, using large number of features in the prediction can lead to complex or noisy models and introduction of unnecessary features. The process of dimensionality reduction helps in reducing the number of input features in the model, and to mitigate overfitting. Principal Component Analysis (or PCA) is one such technique for reducing dimensionality or number of features.

In this exercise, we will cover an example using PathMNIST dataset (from MedMNIST (https://medmnist.com/) and use PCA to conduct dimensionality reduction. 

As a next step, we will also train a multi-class classification model (where the output comprises of more than two classes, as would be the case for binary classification problems).

1. First, let's install medmnist, which would give us access to the code, data and libraries created by the developers of the mednist package, to allow data science enthusiasts to develop insights, and produce predictive models.

In [ ]:
!pip install medmnist

2. Next, we import some libraries required to run the code. We will use torch, an open-source machine learning framework, for this exercise. The input dataset **PathMNIST** is from a project called **MedMNIST**, a benchmark for lightweight biomedical image classification tasks. **PathMNIST** represents data that contains labeled examples depicting colorectal cancer. 

In [ ]:
%matplotlib inline
import IPython.core.display
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sklearn import *
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

3. The data_flag field here helps pick the appropriate dataset. On MedMNIST, you will find 18 datasets, PathMNIST being one of them. 

In [ ]:
data_flag = 'pathmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

In [ ]:
print(train_dataset)
print("===================")
print(test_dataset)

In [ ]:
# visualization through .montage

train_dataset.montage(length=10)

In [ ]:
print(type(train_dataset))
print(type(train_dataset.imgs))
print(train_dataset.imgs.shape)

In [ ]:
print(train_dataset.imgs[1].shape)

In [ ]:
np.where(np.any(np.logical_or(train_dataset.labels == 0, train_dataset.labels == 1, train_dataset.labels == 2), axis=1))

In [ ]:
train_dataset.imgs[[14]]

In [ ]:
train_images_1 = train_dataset.imgs
test_images_1 = test_dataset.imgs

train_labels_1 = train_dataset.labels
test_labels_1 = test_dataset.labels

In [ ]:
train_images_1

In [ ]:
train_labels_1[train_labels_1 == 0] = int(0)
train_labels_1[train_labels_1 == 1] = int(1)
train_labels_1[train_labels_1 == 2] = int(2)
train_labels_1[train_labels_1 == 3] = int(3)
train_labels_1[train_labels_1 == 4] = int(4)
train_labels_1[train_labels_1 == 5] = int(5)
train_labels_1[train_labels_1 == 6] = int(6)
train_labels_1[train_labels_1 == 7] = int(7)
train_labels_1[train_labels_1 == 8] = int(8)

test_labels_1[test_labels_1 == 0] = int(0)
test_labels_1[test_labels_1 == 1] = int(1)
test_labels_1[test_labels_1 == 2] = int(2)
test_labels_1[test_labels_1 == 3] = int(3)
test_labels_1[test_labels_1 == 4] = int(4)
test_labels_1[test_labels_1 == 5] = int(5)
test_labels_1[test_labels_1 == 6] = int(6)
test_labels_1[test_labels_1 == 7] = int(7)
test_labels_1[test_labels_1 == 8] = int(8)

In [ ]:
# Flatten the dataset
nsamples, nx, ny, nz = train_images_1.shape
train_images_1_reshaped = train_images_1.reshape((nsamples,nx*ny*nz))

In [ ]:
# Flatten the dataset for the testing dataset
nsamples, nx, ny, nz = test_images_1.shape
test_images_1_reshaped = test_images_1.reshape((nsamples,nx*ny*nz))

In [ ]:
test_images_1_reshaped

In [ ]:
# standardize the data since PCA is highly sensitive to the scale
scaler = preprocessing.StandardScaler()

scaler.fit_transform(train_images_1_reshaped)

trainX = scaler.transform(train_images_1_reshaped)
testX = scaler.transform(test_images_1_reshaped)

In [ ]:
print(trainX.shape)
print(testX.shape)

In [ ]:
# apply the PCA
pca = decomposition.PCA(n_components = 2)

# fit the training set
proj = pca.fit_transform(trainX)

plt.scatter(proj[:, 0], proj[:, 1], c=list(map(int, train_labels_1)), cmap=plt.cm.jet, s=20)
plt.colorbar()

In [ ]:
# calculate the percentage of explained variance
sum(pca.explained_variance_ratio_)

In [ ]:
# apply the PCA 
# the parameter we insert is 0.9
# this means that we select the number of components such that 
# the amount of variance that needs to be explained is greater than the percentage specified by n_components.
pca = decomposition.PCA(0.9)

# fit the training set 
trainX_90 = pca.fit_transform(trainX)

In [ ]:
pca.n_components_

In [ ]:
compressed_image = pca.inverse_transform(trainX_90)

In [ ]:
compressed_image.shape

In [ ]:
trainX[6].shape

In [ ]:
compressed_image = pca.inverse_transform(trainX_90)

plt.figure(figsize=(8, 4))

# original image
plt.subplot(1, 2, 1)
plt.imshow(trainX[6].reshape(28, 28, 3), cmap=plt.cm.gray,interpolation='nearest')
plt.xlabel('2352 covariates')
plt.title('Original image')

# compressed image
plt.subplot(1, 2, 2)
plt.imshow(compressed_image[6].reshape(28, 28, 3), cmap=plt.cm.gray, interpolation='nearest')
plt.xlabel('185 covariates')
plt.title('90% of explained variance')

In [ ]:
plt.ion()
print('Label: ' + str(train_labels_1[0]))
print('The actual image:')
plt.imshow(train_images_1[0])
plt.show()

In [ ]:
plt.ion()
print('Label: ' + str(train_labels_1[100]))
print('The actual image:')
plt.imshow(train_images_1[100])
plt.show()

In [ ]:
train_images_1.shape

In [ ]:
# Flatten the dataset
nsamples, nx, ny, nz = train_images_1.shape
train_images_1_reshaped = train_images_1.reshape((nsamples,nx*ny*nz))

In [ ]:
# Flatten the dataset for the testing dataset
nsamples, nx, ny, nz = test_images_1.shape
test_images_1_reshaped = test_images_1.reshape((nsamples,nx*ny*nz))

In [ ]:
train_images_1_3 = train_dataset.imgs[[i for i, x in enumerate(np.logical_or(train_dataset.labels == 0, train_dataset.labels == 1, train_dataset.labels == 2)) if x]]
test_images_1_3 = test_dataset.imgs[[i for i, x in enumerate(np.logical_or(test_dataset.labels == 0, test_dataset.labels == 1, test_dataset.labels == 2)) if x]]

train_labels_1_3 = train_dataset.labels[[i for i, x in enumerate(np.logical_or(train_dataset.labels == 0, train_dataset.labels == 1, train_dataset.labels == 2)) if x]]
test_labels_1_3 = test_dataset.labels[[i for i, x in enumerate(np.logical_or(test_dataset.labels == 0, test_dataset.labels == 1, test_dataset.labels == 2)) if x]]

In [ ]:
train_labels_1_3[train_labels_1_3 == 0] = int(0)
train_labels_1_3[train_labels_1_3 == 1] = int(1)
train_labels_1_3[train_labels_1_3 == 2] = int(2)

test_labels_1_3[test_labels_1_3 == 0] = int(0)
test_labels_1_3[test_labels_1_3 == 1] = int(1)
test_labels_1_3[test_labels_1_3 == 2] = int(2)

In [ ]:
# Flatten the dataset
nsamples, nx, ny, nz = train_images_1_3.shape
train_images_1_3_reshaped = train_images_1_3.reshape((nsamples,nx*ny*nz))

In [ ]:
# Flatten the dataset for the testing dataset
nsamples, nx, ny, nz = test_images_1_3.shape
test_images_1_3_reshaped = test_images_1_3.reshape((nsamples,nx*ny*nz))

In [ ]:
# standardize the data since PCA is highly sensitive to the scale
scaler = preprocessing.StandardScaler()

scaler.fit_transform(train_images_1_3_reshaped)

trainX = scaler.transform(train_images_1_3_reshaped)
testX = scaler.transform(test_images_1_3_reshaped)

In [ ]:
import time 

start = time.time()
logreg = linear_model.LogisticRegression(solver = 'lbfgs', max_iter = 10)
logreg.fit(trainX, train_labels_1_3)
stop = time.time()

print("training time  =", stop - start)

# predict from the model
predYtest  = logreg.predict(testX)

# calculate accuracy for testing set
acc      = metrics.accuracy_score(test_labels_1_3, predYtest)
print("test accuracy  =", acc)